In [1]:
import sys

sys.path.append("C:\\Users\\sujee\\pydev\\pyspark_learn_project\\common_utils")

from sparkUtils import get_spark_session
spark = get_spark_session()

path = "C:\\Users\\sujee\\pydev\\pyspark_learn_project\\resources"
flight_path = path + "\\spark_input\\flight_data\\2015-summary.csv"
readme_path = path + "\\spark_input\\README.txt"
my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple The".split(" ")
        

my_coll = spark.sparkContext.parallelize(my_collection, 2)

read_rdd= spark.sparkContext.textFile(readme_path)

#reading plaing readme file
# read_rdd.flatMap(lambda x: x.split(" ")).map(lambda x: (1, x)).filter(lambda x: x[1].lower() == 'user'


row = [('A',),('B',),('C',),('B',)]
word = spark.sparkContext.parallelize(row, 2)



file_rdd = spark.sparkContext.textFile(flight_path)

#file_rdd.take(5)
from functools import reduce
word.flatMap(lambda x: x).map(lambda x: (x,1)).groupByKey().collect()

#.map(lambda x: (x[0], reduce(lambda x,y: x+y, x[1])))\
#.filter(lambda x: x[0] == 'B').collect()




[('C', <pyspark.resultiterable.ResultIterable at 0x1df2fdda370>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x1df2fdda400>),
 ('B', <pyspark.resultiterable.ResultIterable at 0x1df2fdda460>)]

In [2]:
word.flatMap(lambda x: x).map(lambda x: (x,1)).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()

#.map(lambda x: (x[0], reduce(lambda x,y: x+y, x[1])))\
#.filter(lambda x: x[0] == 'B').collect()


[('C', [1]), ('A', [1]), ('B', [1, 1])]

In [3]:
word.flatMap(lambda x: x).map(lambda x: (x,1)).groupByKey()\
         .map(lambda x: (x[0], sum(list(x[1])))).collect()

[('C', 1), ('A', 1), ('B', 2)]

In [4]:
#reading csv file in rdd

file_rdd = spark.sparkContext.textFile(flight_path)
file_df = spark.read.csv(flight_path,header=True)



#find the no of flight from US to INDIA in RDD and structured API


file_df.show()




+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [5]:
from pyspark.sql.functions import col
#find no of flight from Unitest States to India
origin_c = col('ORIGIN_COUNTRY_NAME') == 'United States'
dest_c = col('DEST_COUNTRY_NAME') == 'India'

file_df.select(col("*"), (origin_c & dest_c).alias('check')).filter(col('check') == True).show()

+-----------------+-------------------+-----+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|check|
+-----------------+-------------------+-----+-----+
|            India|      United States|   61| true|
+-----------------+-------------------+-----+-----+



In [6]:

file_rdd_1 = file_rdd.map(lambda x:x.split(","))

file_rdd_2 = file_rdd_1.filter(lambda x: x[0] != 'DEST_COUNTRY_NAME' )\
             .filter(lambda x: x[0] == 'United States' and x[1] == 'India')


file_rdd_2.collect()




[['United States', 'India', '62']]

In [6]:
#my_coll.map(lambda x: x.split(" ")).collect()

my_coll.map(lambda x: (x.lower(), 1)).collect()

keyword = my_coll.keyBy(lambda x: x.lower()[0])


map_value = keyword.mapValues(lambda x: x.upper())

flat_map_value = keyword.flatMapValues(lambda x: x.upper())

flat_map_value.collect()





[('s', 'S'),
 ('s', 'P'),
 ('s', 'A'),
 ('s', 'R'),
 ('s', 'K'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E'),
 ('d', 'D'),
 ('d', 'E'),
 ('d', 'F'),
 ('d', 'I'),
 ('d', 'N'),
 ('d', 'I'),
 ('d', 'T'),
 ('d', 'I'),
 ('d', 'V'),
 ('d', 'E'),
 ('g', 'G'),
 ('g', 'U'),
 ('g', 'I'),
 ('g', 'D'),
 ('g', 'E'),
 (':', ':'),
 ('b', 'B'),
 ('b', 'I'),
 ('b', 'G'),
 ('d', 'D'),
 ('d', 'A'),
 ('d', 'T'),
 ('d', 'A'),
 ('p', 'P'),
 ('p', 'R'),
 ('p', 'O'),
 ('p', 'C'),
 ('p', 'E'),
 ('p', 'S'),
 ('p', 'S'),
 ('p', 'I'),
 ('p', 'N'),
 ('p', 'G'),
 ('m', 'M'),
 ('m', 'A'),
 ('m', 'D'),
 ('m', 'E'),
 ('s', 'S'),
 ('s', 'I'),
 ('s', 'M'),
 ('s', 'P'),
 ('s', 'L'),
 ('s', 'E'),
 ('t', 'T'),
 ('t', 'H'),
 ('t', 'E')]

In [8]:
print(keyword.keys().collect(), keyword.values().collect(),sep="\n")



['s', 't', 'd', 'g', ':', 'b', 'd', 'p', 'm', 's', 't']
['Spark', 'The', 'Definitive', 'Guide', ':', 'Big', 'Data', 'Processing', 'Made', 'Simple', 'The']


In [47]:

#look up for key will return list of values with matched key
keyword.lookup('s')


['Spark', 'Simple']

In [9]:
chars = my_coll.flatMap(lambda x: x.lower())
KVChar = my_coll.map(lambda x: (x, 1))


In [10]:
chars.collect()

['s',
 'p',
 'a',
 'r',
 'k',
 't',
 'h',
 'e',
 'd',
 'e',
 'f',
 'i',
 'n',
 'i',
 't',
 'i',
 'v',
 'e',
 'g',
 'u',
 'i',
 'd',
 'e',
 ':',
 'b',
 'i',
 'g',
 'd',
 'a',
 't',
 'a',
 'p',
 'r',
 'o',
 'c',
 'e',
 's',
 's',
 'i',
 'n',
 'g',
 'm',
 'a',
 'd',
 'e',
 's',
 'i',
 'm',
 'p',
 'l',
 'e',
 't',
 'h',
 'e']

In [50]:
KVChar.countByKey()

defaultdict(int,
            {'Spark': 1,
             'The': 2,
             'Definitive': 1,
             'Guide': 1,
             ':': 1,
             'Big': 1,
             'Data': 1,
             'Processing': 1,
             'Made': 1,
             'Simple': 1})

In [56]:
demo = KVChar.countByKey()


print(type(demo), demo.get('The'))



<class 'collections.defaultdict'> 2


In [60]:
from functools import reduce
#def addFunc():
kvchar_group = KVChar.groupByKey().map(lambda row: (row[0], reduce(lambda x,y: x+y, row[1])))

kvchar_group.collect()

[('Spark', 1),
 ('The', 2),
 ('Made', 1),
 ('Definitive', 1),
 ('Guide', 1),
 (':', 1),
 ('Big', 1),
 ('Data', 1),
 ('Processing', 1),
 ('Simple', 1)]

In [62]:
kvchar_group.filter(lambda x: x[0] == 'The').collect()

[('The', 2)]